# 第8章: 機械学習

本章では，Bo Pang氏とLillian Lee氏が公開しているMovie Review Dataのsentence polarity dataset v1.0を用い，文を肯定的（ポジティブ）もしくは否定的（ネガティブ）に分類するタスク（極性分析）に取り組む．

In [1]:
import random

# 70. データの入手・整形
文に関する極性分析の正解データを用い，以下の要領で正解データ（sentiment.txt）を作成せよ．

1. rt-polarity.posの各行の先頭に"+1 "という文字列を追加する（極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
2. rt-polarity.negの各行の先頭に"-1 "という文字列を追加する（極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
3. 上述1と2の内容を結合（concatenate）し，行をランダムに並び替える

sentiment.txtを作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ

In [2]:
# nkfコマンドでバイナリからutf8に変換
with open('rt-polaritydata/pos', 'r') as f:
    sentiment_pos = ["+1 " + x[:-1] for x in f.readlines()]
with open('rt-polaritydata/neg', 'r') as f:
    sentiment_neg = ["-1 " + x[:-1] for x in f.readlines()]
sentiment_pos[0] = sentiment_pos[0].replace("\ufeff","")
sentiment_neg[0] = sentiment_neg[0].replace("\ufeff","")
sentiment = sentiment_pos + sentiment_neg
random.shuffle(sentiment)
with open('sentiment.txt', 'w') as f:
    f.writelines([x + "\n" for x in sentiment])

In [3]:
with open('sentiment.txt', 'r') as f:
    sentiment = [x[:-1] for x in f.readlines()]
    count = len(sentiment)
    count_pos = len([x for x in sentiment if x[:2] == "+1"])
    count_neg = len([x for x in sentiment if x[:2] == "-1"])
count, count_pos, count_neg

(10661, 5330, 5331)

# 71. ストップワード
英語のストップワードのリスト（ストップリスト）を適当に作成せよ．さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，それ以外は偽を返す関数を実装せよ．さらに，その関数に対するテストを記述せよ．

In [21]:
with open('stopwords.csv', 'r') as f:
    stopwords = [x[:-1] for x in f.readlines()]
def validate(word):
    return word in stopwords

In [13]:
validate('you')

True

In [14]:
validate('aaaaa')

False

# 72. 素性抽出
極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．

In [50]:
from stemming.porter2 import stem
wordlist = []
for line in sentiment:
    wordlist.extend([stem(x) for x in line[:-1].split(" ")[1:]  if not validate(x)])
    column = set(wordlist)

In [51]:
import pandas as pd
bow = pd.DataFrame(columns=column)
bow

,sixties-styl,power-lunch,translat,unambiti,self-piti,vibranc,fundada,musi,hank,portraitur,...,know-how,twister,jolli,paradiso,ricochet,ghast,akin,gotta,bai,gown


# 73. 学習
72で抽出した素性を用いて，ロジスティック回帰モデルを学習せよ．

In [39]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()